In [1]:
!pip install pymssql


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from joblib import load 
from sqlalchemy import create_engine
import pymssql
import pandas as pd
import plotly.graph_objects as go

In [ ]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [ ]:
data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m, siccd
    from data
    where date='2022-01'
    """, 
    conn
)
data = data.dropna()

In [ ]:
forest = load("forest2.joblib")

X = data[["roeq", "mom12m"]]
data["predict"] = forest.predict(X)

In [ ]:
data = data.sort_values(
  by="predict",
  ascending=False
)
data = data.reset_index(drop=True)

data.head(3)

In [ ]:
data.tail(3)

In [ ]:
data["roeq"] = data.roeq.rank(pct=True)
data["mom12m"] = data.mom12m.rank(pct=True)
data.head(3)

In [ ]:
data.tail(3)

In [ ]:

top = data.iloc[:100]
bottom = data.iloc[-100:]

trace1 = go.Box(
  y=top.roeq,
  name="top"
)
trace2 = go.Box(
  y=bottom.roeq,
  name="bottom"
)
fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)

fig.update_layout(
    template="plotly_dark",
    yaxis_tickformat=".0%",
)

fig.show()

In [ ]:
top["type"] = "top"
bottom["type"] = "bottom"
data2 = pd.concat((top, bottom))

import plotly.express as px
fig = px.histogram(data2, x="roeq", color="type")
fig.update_layout(
    template="plotly_dark",
)

fig.show()

In [ ]:
trace1 = go.Box(
  y=top.mom12m,
  name="top"
)
trace2 = go.Box(
  y=bottom.mom12m,
  name="bottom"
)
fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)

fig.update_layout(
    template="plotly_dark",
    yaxis_tickformat=".0%",
)

fig.show()

In [ ]:
fig = px.histogram(data2, x="mom12m", color="type")
fig.update_layout(
    template="plotly_dark",
)
fig.show()